In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
import numpy as np

In [5]:
#Loading in the movies dataset
movies_df = pd.read_csv('../ml-1m/movies.dat', sep='::', header=None, engine='python')
movies_df.columns = ['MovieID', 'Title', 'Genres']
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
#Loading in the ratings dataset
ratings_df = pd.read_csv('../ml-1m/ratings.dat', sep='::', header=None, engine='python')
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
user_rating_df = ratings_df.pivot(index='UserID', columns='MovieID', values='Rating')
user_rating_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
norm_user_rating_df = user_rating_df.fillna(0) / 5.0
trainX = norm_user_rating_df.values
trainX[0:5, 0:10]

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0. , 0. , 0. ]])

In [27]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(torch.FloatTensor(trainX), batch_size=batch_size, shuffle=True)

In [28]:
example = next(iter(train_loader))

In [29]:
print(example.shape)
print(example[0].shape)

torch.Size([64, 3706])
torch.Size([3706])


In [42]:
class RBM(nn.Module):
   def __init__(self,
               n_vis=3706,
               n_hin=500,
               k=5):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hin,n_vis)*1e-2)
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hin))
        self.k = k
    
   def sample_from_p(self,p):
       return F.relu(torch.sign(p - Variable(torch.rand(p.size()))))   # draw binary samples with p being the probablity of being 1, we can also use bernoulli, this implementation is just more bottom.
    
   def v_to_h(self,v):
        p_h = torch.sigmoid(F.linear(v,self.W,self.h_bias))
        sample_h = self.sample_from_p(p_h)
        return p_h,sample_h
    
   def h_to_v(self,h):
        p_v = torch.sigmoid(F.linear(h,self.W.t(),self.v_bias))
        sample_v = self.sample_from_p(p_v)
        return p_v,sample_v
        
   def forward(self,v):
        p_h1,h1 = self.v_to_h(v)
        
        h_ = h1
        for _ in range(self.k):
            p_v_,v_ = self.h_to_v(h_)
            p_h_,h_ = self.v_to_h(v_)
        
        return v,v_, p_v_
    
   def free_energy(self,v):
        vbias_term = v.mv(self.v_bias) # torch.mv() performs a matrix-vector product
        wx_b = F.linear(v,self.W,self.h_bias)
        hidden_term = wx_b.exp().add(1).log().sum(1)
        return (-hidden_term - vbias_term).mean()

In [43]:
rbm = RBM(k=1)
train_op = optim.SGD(rbm.parameters(),0.1)

for epoch in range(10):
    loss_ = []
    for _, data in enumerate(train_loader):
        # data = Variable(data.view(-1,784))  # 784 = 1 * 28 * 28
        # sample_data = data.bernoulli()   # draw bernoulli variables using the probability in the tensor, input should be in the range of [0, 1].
        
        v,v1,_ = rbm(data)
        loss = rbm.free_energy(v) - rbm.free_energy(v1)
        loss_.append(loss.data)
        train_op.zero_grad()
        loss.backward()
        train_op.step()

    print("Training loss for {} epoch: {}".format(epoch, np.mean(loss_)))

Training loss for 0 epoch: -111.7513656616211
Training loss for 1 epoch: -45.5367546081543
Training loss for 2 epoch: -22.589345932006836
Training loss for 3 epoch: -13.990769386291504
Training loss for 4 epoch: -9.718010902404785
Training loss for 5 epoch: -7.415131568908691
Training loss for 6 epoch: -5.953097343444824
Training loss for 7 epoch: -4.648928642272949
Training loss for 8 epoch: -4.907916069030762
Training loss for 9 epoch: -3.8093464374542236


# Inference

## Check an example

In [68]:
mock_user_id = 215
inputUser = torch.FloatTensor(trainX[mock_user_id - 1]).reshape(1,-1)
print(inputUser.shape)
print(example.shape)
print(inputUser)

torch.Size([1, 3706])
torch.Size([64, 3706])
tensor([[0.8000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [48]:
v, v1, p_v1 = rbm(inputUser)
print(v)
print(v1)
print(p_v1)
print(p_v1.shape)

tensor([[0.8000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward0>)
tensor([[0.0352, 0.0351, 0.0279,  ..., 0.0006, 0.0006, 0.0059]],
       grad_fn=<SigmoidBackward>)
torch.Size([1, 3706])


In [60]:
scored_movies_df_mock = movies_df[movies_df['MovieID'].isin(user_rating_df.columns)]
scored_movies_df_mock['RecommendationScore'] = p_v1.squeeze().detach().numpy()
scored_movies_df_mock.sort_values(['RecommendationScore'], ascending=False).head()
# scored_movies_df_mock

<ipython-input-60-b059be3d52e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scored_movies_df_mock['RecommendationScore'] = p_v1.squeeze().detach().numpy()


,MovieID,Title,Genres,RecommendationScore
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi,0.928799
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,0.924716
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,0.900216
3106,3175,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi,0.871700
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,0.828720


In [65]:
ratings_df_mock = ratings_df[ratings_df['UserID'] == mock_user_id]
merged_df_mock = scored_movies_df_mock.merge(ratings_df_mock, on = 'MovieID', how = 'outer')
merged_df_mock.sort_values(['RecommendationScore'], ascending=False, inplace=True)
merged_df_mock.head(20)

,MovieID,Title,Genres,RecommendationScore,UserID,Rating,Timestamp
466,480,Jurassic Park (1993),Action|Adventure|Sci-Fi,0.928799,215.0,5.0,976899784.0
253,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,0.924716,215.0,5.0,976899190.0
2426,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,0.900216,215.0,5.0,976908635.0
2958,3175,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi,0.871700,NaN,NaN,NaN
1106,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,0.828720,NaN,NaN,NaN
2374,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,0.750407,NaN,NaN,NaN
1485,1617,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller,0.488800,215.0,4.0,976908485.0
1104,1193,One Flew Over the Cuckoo's Nest (1975),Drama,0.480483,NaN,NaN,NaN
1050,1127,"Abyss, The (1989)",Action|Adventure|Sci-Fi|Thriller,0.471493,NaN,NaN,NaN
31,32,Twelve Monkeys (1995),Drama|Sci-Fi,0.406044,NaN,NaN,NaN
